In [1]:
import sqlparse

from cast.cache import PostData

from django.test.client import RequestFactory
from django.db import connection, reset_queries

# Measure PostData Cache Performance

Find out which pages generate queries.

In [2]:
def show_queries(queries):
    for query in queries:
        formatted_sql = sqlparse.format(query['sql'], reindent=True, keyword_case='upper')
        print(formatted_sql)
        

def blocker(*args):
    raise Exception("No database access allowed here.")

In [3]:
# blog_slug = "ephes_blog"
blog_slug = "das_claas_und_nora_blog"
blog = Blog.objects.get(slug=blog_slug)
request_factory = RequestFactory()

# Just measure for a list of Posts

In [4]:
%%time
reset_queries()
post_queryset = Post.objects.live().descendant_of(blog).order_by("-visible_date")
post_data = PostData.create_from_post_queryset(
    request=request_factory.get("/"),
    blog=blog,
    post_queryset=post_queryset,
    template_base_dir="bootstrap4",
)
print(len(connection.queries))

308
CPU times: user 664 ms, sys: 22.3 ms, total: 687 ms
Wall time: 740 ms


In [5]:
post = Post.objects.get(pk=501)

In [6]:
%%time
request = request_factory.get("/foobar/")
posts = list(post_queryset)
reset_queries()
# request = request_factory.get(post_data.page_url_by_id[post.pk])
for post in posts:
    # print("post pk: ", post.pk)
    with connection.execute_wrapper(blocker):
        post._local_template_name = "post_body.html"
        response = post.serve(request, post_data=post_data).render()

page url:  /blogs/das_claas_und_nora_blog/marz-2024-03-18/
page url:  /blogs/das_claas_und_nora_blog/marz-2024-03-11/
page url:  /blogs/das_claas_und_nora_blog/marz-2024-03-04/
page url:  /blogs/das_claas_und_nora_blog/februar-2024-02-26/
page url:  /blogs/das_claas_und_nora_blog/februar-2024-02-19/
page url:  /blogs/das_claas_und_nora_blog/februar-2024-02-12/
page url:  /blogs/das_claas_und_nora_blog/februar-2024-02-05/
page url:  /blogs/das_claas_und_nora_blog/januar-2024-01-29/
page url:  /blogs/das_claas_und_nora_blog/ginos-kinderland/
page url:  /blogs/das_claas_und_nora_blog/januar-2024-01-22/
page url:  /blogs/das_claas_und_nora_blog/januar-2024-01-15/
page url:  /blogs/das_claas_und_nora_blog/januar-2024-01-08/
page url:  /blogs/das_claas_und_nora_blog/januar-2024-01-01/
page url:  /blogs/das_claas_und_nora_blog/neujahr-20232024/
page url:  /blogs/das_claas_und_nora_blog/karussell-auf-dem-weihnachtsmarkt/
page url:  /blogs/das_claas_und_nora_blog/dezember-2023-12-25/
page url: 

In [7]:
print(len(posts))

315


In [8]:
print(len(connection.queries))

0


In [9]:
show_queries(connection.queries)

# Feed Performance

In [10]:
from cast.feeds import LatestEntriesFeed

In [11]:
%%time
reset_queries()
post_queryset = Post.objects.live().descendant_of(blog).order_by("-visible_date")
post_data = PostData.create_from_post_queryset(
    request=request_factory.get("/"),
    blog=blog,
    post_queryset=post_queryset,
    template_base_dir="bootstrap4",
)
print(len(connection.queries))

10
CPU times: user 416 ms, sys: 6.86 ms, total: 422 ms
Wall time: 439 ms


In [12]:
# call this cell once without blocker to populate SITE_CACHE
response = LatestEntriesFeed(post_data=post_data)(request, slug=blog_slug)

page url:  https://wersdoerfer.de/blogs/das_claas_und_nora_blog/marz-2024-03-18/
page url:  https://wersdoerfer.de/blogs/das_claas_und_nora_blog/marz-2024-03-11/
page url:  https://wersdoerfer.de/blogs/das_claas_und_nora_blog/marz-2024-03-04/
page url:  https://wersdoerfer.de/blogs/das_claas_und_nora_blog/februar-2024-02-26/
page url:  https://wersdoerfer.de/blogs/das_claas_und_nora_blog/februar-2024-02-19/
page url:  https://wersdoerfer.de/blogs/das_claas_und_nora_blog/februar-2024-02-12/
page url:  https://wersdoerfer.de/blogs/das_claas_und_nora_blog/februar-2024-02-05/
page url:  https://wersdoerfer.de/blogs/das_claas_und_nora_blog/januar-2024-01-29/
page url:  https://wersdoerfer.de/blogs/das_claas_und_nora_blog/ginos-kinderland/
page url:  https://wersdoerfer.de/blogs/das_claas_und_nora_blog/januar-2024-01-22/
page url:  https://wersdoerfer.de/blogs/das_claas_und_nora_blog/januar-2024-01-15/
page url:  https://wersdoerfer.de/blogs/das_claas_und_nora_blog/januar-2024-01-08/
page ur

In [13]:
%%time
request = request_factory.get("/foobar/")
reset_queries()

# with connection.execute_wrapper(blocker):
response = LatestEntriesFeed(post_data=post_data)(request, slug=blog_slug)

page url:  https://wersdoerfer.de/blogs/das_claas_und_nora_blog/marz-2024-03-18/
page url:  https://wersdoerfer.de/blogs/das_claas_und_nora_blog/marz-2024-03-11/
page url:  https://wersdoerfer.de/blogs/das_claas_und_nora_blog/marz-2024-03-04/
page url:  https://wersdoerfer.de/blogs/das_claas_und_nora_blog/februar-2024-02-26/
page url:  https://wersdoerfer.de/blogs/das_claas_und_nora_blog/februar-2024-02-19/
page url:  https://wersdoerfer.de/blogs/das_claas_und_nora_blog/februar-2024-02-12/
page url:  https://wersdoerfer.de/blogs/das_claas_und_nora_blog/februar-2024-02-05/
page url:  https://wersdoerfer.de/blogs/das_claas_und_nora_blog/januar-2024-01-29/
page url:  https://wersdoerfer.de/blogs/das_claas_und_nora_blog/ginos-kinderland/
page url:  https://wersdoerfer.de/blogs/das_claas_und_nora_blog/januar-2024-01-22/
page url:  https://wersdoerfer.de/blogs/das_claas_und_nora_blog/januar-2024-01-15/
page url:  https://wersdoerfer.de/blogs/das_claas_und_nora_blog/januar-2024-01-08/
page ur

In [14]:
print(len(connection.queries))

0


In [15]:
show_queries(connection.queries)

# Render A Blog Index Page

In [16]:
import pickle

from cast.models import Blog
from cast.cache import PagedPostData

In [17]:
%%time
reset_queries()
post_queryset = blog.unfiltered_published_posts
post_data = PagedPostData.create_from_blog_index_request(
    request=request_factory.get("/"),
    blog=blog,
)
print(len(connection.queries))

22
CPU times: user 26.6 ms, sys: 6.32 ms, total: 32.9 ms
Wall time: 41.6 ms


In [18]:
cache_path = "paged_post_data.pkl"
with open(cache_path, "wb") as f:
    pickle.dump(post_data, f)

AttributeError: Can't pickle local object 'BaseCSVFilter.__init__.<locals>.ConcreteCSVField'

In [ ]:
with open(cache_path, "rb") as f:
    loaded = pickle.load(f)

In [19]:
%%time
request = request_factory.get(blog.get_url())
request.htmx = False
reset_queries()

# with connection.execute_wrapper(blocker):
response = blog.serve(request, post_data=post_data)

CPU times: user 1.41 ms, sys: 204 µs, total: 1.61 ms
Wall time: 1.62 ms


In [20]:
print(len(connection.queries))

0


In [21]:
show_queries(connection.queries)